In [2]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core import tools
from langchain.tools import tool
from IPython.display import Markdown


load_dotenv("./.env")

llm = ChatOpenAI(model="gpt-4o")



In [3]:
@tool 
def write_email(to: str, subject:str, content: str) -> str:
    """Write and send email"""
    return f"Email sent to {to} with subject {subject} and content : {content}"

model_with_tools = llm.bind_tools( [write_email] , parallel_tool_calls=False)
output = model_with_tools.invoke("Envia un email a manuonda@gmail.com diciendo que es un crack")

print(output.tool_calls[0])
print(f"Name tool : {output.tool_calls[0]['name']}")

type(output)
args = output.tool_calls[0]['args']
print(f"Args : {args}")

#call the tool
result = write_email.invoke(args)
Markdown(result)


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************Iz0A. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [14]:
from typing import TypedDict
from langgraph.graph import StateGraph, START, END

class StateSchema(TypedDict):
    request: str
    email: str

workflow = StateGraph(StateSchema)




def write_email_node(state: StateSchema) -> StateSchema:
    # Imperative code that processes the request
    output = model_with_tools.invoke(state["request"])
    print(output)
    if output.tool_calls:
      args = output.tool_calls[0]['args']
      email = write_email.invoke(args)
      return {"email": email}
    else:
        return {"email": output.content}



workflow = StateGraph(StateSchema)
workflow.add_node("write_email_node", write_email_node)
workflow.add_edge(START, "write_email_node")
workflow.add_edge("write_email_node", END)

app = workflow.compile()

app.invoke({"request": "Draft a response to my boss (boss@company.ai) about tomorrow's meeting"})#


content='Of course! Could you please provide more details about what you would like to convey in your response? For example, are you confirming attendance, asking for specific agenda topics, or requesting a change in time or format?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 66, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_07871e2ad8', 'finish_reason': 'stop', 'logprobs': None} id='run-f67c36be-b38b-44af-9d02-cf624a7b97b5-0' usage_metadata={'input_tokens': 66, 'output_tokens': 44, 'total_tokens': 110, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


{'request': "Draft a response to my boss (boss@company.ai) about tomorrow's meeting",
 'email': 'Of course! Could you please provide more details about what you would like to convey in your response? For example, are you confirming attendance, asking for specific agenda topics, or requesting a change in time or format?'}

In [15]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=llm,
    tools=[write_email],
    prompt="Respond to the user's request using the tools provided"
)
result = agent.invoke(
    {"messages": [{"role": "user", "content": "Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!"}]}
 )
for  m in result["messages"]:
    m.pretty_print()
    

================================ Human Message =================================

Draft a response to my boss (boss@company.ai) confirming that I want to attend Interrupt!
================================== Ai Message ==================================
Tool Calls:
  write_email (call_g5kwOOJtukP84XR0FY1hh0wT)
 Call ID: call_g5kwOOJtukP84XR0FY1hh0wT
  Args:
    to: boss@company.ai
    subject: Confirmation: Attendance at Interrupt! Conference
    content: Hi Boss,

I hope this message finds you well. I am writing to confirm that I would like to attend the Interrupt! conference. I believe it will be a great opportunity to gain insights and bring back valuable knowledge to our team.

Please let me know if there are any specific sessions or topics you would like me to focus on during the conference.

Thank you for supporting my attendance.

Best,
[Your Name]
================================= Tool Message =================================
Name: write_email

Email sent to boss@company.ai wit

#Persistence 
Threads

It can be very useful to allow agents to pause during long running tasks.

LangGraph has a built-in persistence layer, implemented through checkpointers, to enable this.

When you compile graph with a checkpointer, the checkpointer saves a checkpoint of the graph state at every step.

Checkpoints are saved to a thread, which can be accessed after graph execution completes.

In [1]:
from langgraph.checkpoint import InMemorySaver
agent = create_react_agent(
    tools=[write_email],
    llm = llm,
    prompt="Responde to the user's request using the tools provided.",
    checkpointer=InMemorySaver
)

config = {"configurable": {"thread_id":"1"}}
result = agent.invoke({"messages":[{"role": "user", "content": "What are somge doog practices for writing emails ? "}]}, config)

#Get th latest state snapshot 
config = {"configurable": {"thread_id": "1"}}
state = agent.get_state(config)
for m in state.values["messages"]:
   m.pretty_print()


ImportError: cannot import name 'InMemorySaver' from 'langgraph.checkpoint' (unknown location)